In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
#import statsmodels.api as sm
import math

# Loading and transforming the data


In [ ]:
df = pd.read_csv("SAheart.txt",delim_whitespace=True)
df.loc[df["famhist"] == "Present", "famhist"] = 1
df.loc[df["famhist"] == "Absent", "famhist"] = 0
df.head()

# Splitting data into independet and dependent


In [ ]:
Y = df["chd"]
X = df.drop(["chd"],axis=1)
X = (X - X.mean(axis=0))/X.std(axis=0) #standardizing the independent variables
X = X.astype(float).to_numpy()
Y = Y.astype(float).to_numpy()
p = X.shape[1]
n = X.shape[0]
Y = Y.reshape((n, 1))

X_1 = np.ones(n, dtype=int) #intercept
X_1 = np.reshape(X_1, (n, 1))
X = np.hstack((X_1, X)) #gaussian covariates

In [ ]:
#Y = df["chd"]
#X = df.drop(["chd"],axis=1)
#X = (X - X.mean(axis=0))/X.std(axis=0) #standardizing the independent variables
#X = sm.add_constant(X) # add constant vector for the intercept

# Reference model (uses Statsmodel) -- jusk for checking our results

In [ ]:
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit

model = Probit(Y, X.astype(float))
probit_model = model.fit()
print(probit_model.summary())

# Probit with Fisher Scoring

In [ ]:
def probit(X,Y, epsilon):
#X = covariates
#Y = dependent variable
#epsilon = threshold for convergence

    n,p = np.shape(X)

    #Initial values of the algorithms
    b_0 = np.zeros((p,1))
    eta_0 = np.dot(X, b_0)
    mu_0 = norm.cdf(eta_0)

    W = np.diag((((norm.pdf(eta_0))**2)/(mu_0*(1-mu_0))).reshape(n,))
    Z = eta_0+(Y-mu_0)/(norm.pdf(eta_0))


    #boolean
    convergence = False
    while not convergence:

        #new estimate
        b = np.linalg.multi_dot([np.linalg.inv(np.linalg.multi_dot([X.T, W, X])), X.T,W, Z])
        eta = np.dot(X, b)
        mu = norm.cdf(eta)
        W = np.diag((((norm.pdf(eta))**2)/(mu*(1-mu))).reshape(n,))
        Z = eta+(Y-mu)/(norm.pdf(eta))

        if np.linalg.norm(b-b_0)/(np.linalg.norm(b_0)+epsilon) < epsilon:
        #relative convergence criterion
            convergence = True
            print("\nConvergence reached with value:",np.linalg.norm(b-b_0)/(np.linalg.norm(b_0)+epsilon))
        b_0 = b #old value

    return(b)

# Fisher Scoring model output

In [ ]:
beta = probit(X,Y,epsilon = 0.000001)
#print("Beta:", beta.reshape((p+1,))) #estimated beta
#our own step by step model
betas = pd.DataFrame(beta)
betas.rename(columns={0: 'Betas'},inplace=True)
new_index = list(df.columns)
betas.rename(index=dict(zip(betas.index, new_index)),inplace=True)
betas

# Metropolis Hastings model

# Define Posterior

In [ ]:
# Normal CDF: P(c <= XB)
def normal_cdf(X, beta): #accepts vectors too
    linear_predictor = np.array(X) @ np.array(beta)
    #print(linear_predictor)
    prob_vector = []
    if isinstance(linear_predictor,float):
        return 1 / 2 * (1 + np.math.erf(linear_predictor / np.sqrt(2)))
    else:
        for lin in linear_predictor:
            prob = 1 / 2 * (1 + np.math.erf(lin / np.sqrt(2)))
            prob_vector.append(prob)
        return np.array(prob_vector)

# Prior function
#def prior(beta):
#    return 1 #non informative prior

def prior_gaussian(beta, scale=10):
    # Calculate the probability density function of a normal distribution
    pdf_values = (1 / (np.sqrt(2 * np.pi) * scale)) * np.exp(-0.5 * (beta / scale)**2)

    # Compute the product of the probabilities for each parameter
    prior_prob = np.prod(pdf_values)
    return prior_prob

def prior_uniform(beta, scale=10):
    return 1

def cauchy_pdf(x, loc=0, scale=1):
    # Cauchy probability density function
    return (1 / (np.pi * scale * (1 + ((x - loc) / scale)**2)))

def prior_cauchy(beta, scale=10):
    # Calculate the probability density function of a Cauchy distribution
    pdf_values = cauchy_pdf(beta, loc=0, scale=scale)

    # Compute the product of the probabilities for each parameter
    prior_prob = np.prod(pdf_values)

    return prior_prob

#Log Likelyhood function -- not used in the Metroposlis Hastings
#def log_likelihood(y, X, beta): #log_likelyhood requires sum of logs
#    log_likelihood_term = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))
#    return log_likelihood_term

# Likelihood function
def likelihood(y, X, beta):
    p = normal_cdf(X, beta)
    likelihood_term = np.prod(p**y * (1 - p)**(1 - y))
    return likelihood_term

# Posterior function
def posterior(beta, y, X, prior_type = 'uniform'):
    if prior_type == 'uniform':
      prior = prior_uniform
    if prior_type == 'normal':
      prior = prior_gaussian
    if prior_type == 'cauchy':
      prior = prior_cauchy
    post = likelihood(y, X, beta) * prior(beta)
    if post != 0:
        return post
    else:
        return 0.0001

In [ ]:
# Metropolis-Hastings algorithm
def metropolis_hastings(initial_beta, iterations, y, X, prior_type):
    beta_samples = [initial_beta]

    for _ in range(iterations):
        current_beta = beta_samples[-1]

        # Propose a new beta using a random walk
        proposal = current_beta + np.random.normal(0, 0.05, size=len(current_beta))

        # Calculate acceptance ratio
        acceptance_ratio = min(1, posterior(proposal, y, X, prior_type) / posterior(current_beta, y, X, prior_type))

        # Accept or reject the proposed beta
        if np.random.rand() < acceptance_ratio:
            beta_samples.append(proposal)
        else:
            beta_samples.append(current_beta)

    return np.array(beta_samples)

# Run Metropolis Hastings

In [ ]:
#set starting value of beta to a vector of zeros
init_beta = np.zeros(10) #np.zeros(10) + 0.01 #if working with logs
# RUN metropolis hastings
m = metropolis_hastings(init_beta, 10000, Y, X, 'normal')

In [ ]:
m[:-10]

In [ ]:
#Mean of the last 100 sampled betas
betas_100mean = np.mean(m[:-100],axis=0)
betas_100mean

In [ ]:
#set starting value of beta to a vector of zeros
init_beta = np.zeros(10) #np.zeros(10) + 0.01 #if working with logs
# RUN metropolis hastings
m_unif = metropolis_hastings(init_beta, 10000, Y, X, 'uniform')

In [ ]:
#Mean of the last 100 sampled betas
betas_100mean_unif = np.mean(m_unif[:-100],axis=0)
betas_100mean_unif

In [ ]:
#set starting value of beta to a vector of zeros
init_beta = np.zeros(10) #np.zeros(10) + 0.01 #if working with logs
# RUN metropolis hastings
m_cauchy = metropolis_hastings(init_beta, 10000, Y, X, 'uniform')

In [ ]:
#Mean of the last 100 sampled betas
betas_100mean_cauchy = np.mean(m_cauchy[:-100],axis=0)
betas_100mean_cauchy

In [ ]:
# Display estimated betas
betas_estimate = pd.DataFrame(betas_100mean)
betas_estimate.rename(columns={0: 'Mean of last 100 Betas'},inplace=True)
new_index = list(df.columns)
betas_estimate.rename(index=dict(zip(betas_estimate.index, new_index)),inplace=True)
betas_estimate

In [ ]:
#Convergence plots for the betas
plt.plot(m_unif)
plt.xlabel('Iterations')
plt.ylabel('Parameter Value')
plt.title('Convergence Plot for the 10 Beta Parameters')
plt.show()

In [ ]:
fig, axes = plt.subplots(10, 1, figsize=(10, 2*10))

# Loop through variables and plot ACF for each beta
for j in range(10):
    sm.graphics.tsa.plot_acf(m[:, j], lags=40, ax=axes[j], title=f'ACF for Variable {j + 1}')

plt.tight_layout()
plt.show()

In [ ]:
#Convergence as shown by ACF with thinning every 10 steps

betas_burnin = m[1000:]
fig, axes = plt.subplots(10, 1, figsize=(10, 2*10))

# Loop through variables and plot ACF for each beta
for j in range(10):
    sm.graphics.tsa.plot_acf(betas_burnin[:, j], lags=40, ax=axes[j], title=f'ACF for Variable {j + 1}')

plt.tight_layout()
plt.show()

In [ ]:
#Convergence as shown by ACF with thinning every 10 steps

betas_thin = m[1::10]
fig, axes = plt.subplots(10, 1, figsize=(10, 2*10))

# Loop through variables and plot ACF for each beta
for j in range(10):
    sm.graphics.tsa.plot_acf(betas_thin[:, j], lags=40, ax=axes[j], title=f'ACF for Variable {j + 1}')

plt.tight_layout()
plt.show()